In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from waterbirds import data_builder
from shared import train
from shared.utils import restrict_GPU_tf
import shared.train as sht
import os 
import tensorflow as tf
import re

In [4]:
p_tr = 0.8
py1_y0 = 0.5
py1_y0_shift_list = [0.1, 0.5,  0.8]
pflip0 = 0.01
pflip1 = 0.01
oracle_prop = 0.0
exp_dir = '/data/ddmg/slabs/dummy/'
num_epochs = 10 
batch_size = 128
training_steps = 1
alpha = 0.0 
sigma = 0.1 
dropout_rate = 0.0
l2_penalty = 0.0 
embedding_dim = 1000 
random_seed = 0 
cleanup = False 
gpuid = '7'
weighted_mmd = 'unweighted'
architecture = 'pretrained_resnet'
pixel = 64

In [5]:
def dataset_builder():
    return data_builder.build_input_fns(
        p_tr=p_tr,
        py1_y0=py1_y0,
        py1_y0_s=py1_y0_shift_list,
        pflip0=pflip0,
        pflip1=pflip1,
        oracle_prop=oracle_prop,
        random_seed=random_seed)

restrict_GPU_tf(gpuid)


Using GPU:7 with 0% of the memory


In [6]:
scratch_exp_dir = exp_dir.replace('/data/ddmg/slabs/',
    '/data/scratch/mmakar/')
if not os.path.exists(scratch_exp_dir):
    os.mkdir(scratch_exp_dir)

sht.cleanup_directory(scratch_exp_dir)

params = {
    "pixel": pixel,
    "training_steps": training_steps,
    "architecture": architecture,
    "batch_size": batch_size,
    "alpha": alpha,
    "sigma": sigma,
    "weighted_mmd": weighted_mmd,
    "dropout_rate": dropout_rate,
    "l2_penalty": l2_penalty,
    "embedding_dim": embedding_dim,
    "label_ind": 0
}

run_config = tf.estimator.RunConfig(
    tf_random_seed=random_seed,
    save_checkpoints_secs=300,
    keep_checkpoint_max=2)
est = tf.estimator.Estimator(
    sht.model_fn, model_dir=scratch_exp_dir,
    params=params, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': '/data/scratch/mmakar/dummy/', '_tf_random_seed': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
input_fns = dataset_builder()
train_input_fn, valid_input_fn, eval_input_fn_creater = input_fns
sht.cleanup_directory(scratch_exp_dir)
est.train(train_input_fn, steps=training_steps)
validation_results = est.evaluate(valid_input_fn)

if weighted_mmd == "True":
    validation_results['loss'] = validation_results['weighted_loss']
validation_results.pop('weighted_loss', None)
validation_results.pop('mean_pred', None)

all_results = {"validation": validation_results}

if py1_y0_shift_list is not None:
    for py in py1_y0_shift_list:
        eval_input_fn = eval_input_fn_creater(py)
        distribution_results = est.evaluate(eval_input_fn, steps=1e5)
        print("######################################")
        print(py, distribution_results['mean_pred'], distribution_results['mean_class'])
        print("######################################")
        distribution_results.pop('weighted_loss', None)
        # distribution_results.pop('mean_pred', None)
        all_results[f'shift_{py}'] = distribution_results
            
            
# sht.cleanup_directory(scratch_exp_dir)


/data/ddmg/users/mmakar/shared_conda/anaconda3/envs/slabs/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/data/ddmg/users/mmakar/shared_conda/anaconda3/envs/slabs/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/data/ddmg/slabs/structured_labels/waterbirds/data_builder.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

(7830, 6)
(1958, 6)


/data/ddmg/slabs/structured_labels/waterbirds/data_builder.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_data_frame.background_filename[(
/data/ddmg/slabs/structured_labels/waterbirds/data_builder.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_data_frame.background_filename[(


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /data/scratch/mmakar/dummy/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1...
INFO:tensorflow:Saving checkpoints for 1 into /data/scratch/mmakar/dummy/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1...
INFO:tensorflow:Loss for final step: None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting eva

TypeError: eval_input_fn_creater() missing 1 required positional argument: 'params'

In [8]:
if py1_y0_shift_list is not None:
    for py in py1_y0_shift_list:
        eval_input_fn = eval_input_fn_creater(py, params)
        distribution_results = est.evaluate(eval_input_fn, steps=1e5)
        print("######################################")
        print(py, distribution_results['mean_pred'], distribution_results['mean_class'])
        print("######################################")
        distribution_results.pop('weighted_loss', None)
        # distribution_results.pop('mean_pred', None)
        all_results[f'shift_{py}'] = distribution_results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-01T10:48:52Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/scratch/mmakar/dummy/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 74.65718s
INFO:tensorflow:Finished evaluation at 2020-12-01-10:50:07
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.499, global_step = 1, loss = 0.6930573, mean_class = 0.0, mean_pred = 0.49478206, mmd = 0.07169962, weighted_loss = 0.6929819
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1: /data/scratch/mmakar/dummy/model.ckpt-1
######################################
0.1 0.49478206 0.0
######################################
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-01T10:50:14Z
INFO:tensorflow:Graph was finalized.
IN

In [15]:
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import dtypes
def serving_input_fn():
    feat = array_ops.placeholder(dtype=dtypes.float32)
    return tf.estimator.export.TensorServingInputReceiver(features=feat, receiver_tensors=feat)


In [30]:
est.export_saved_model(f'{exp_dir}saved_model', serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['classify', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /data/scratch/mmakar/dummy/model.ckpt-1
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /data/ddmg/slabs/dummy/saved_model/temp-1606840969/saved_model.pb


b'/data/ddmg/slabs/dummy/saved_model/1606840969'

In [31]:
# identify latest TODO 
# from tensorflow.contrib import predictor
loaded = tf.saved_model.load('/data/ddmg/slabs/dummy/saved_model/1606840969')
infer = loaded.signatures["serving_default"]


In [33]:
eval_ds = eval_input_fn()

In [47]:
preds = []
labs = []
for batch_id, examples in enumerate(eval_ds):
    print(f'-----{batch_id}/{len(eval_ds)}----')
    x, y = examples
    preds.append(infer(x)['probabilities'].numpy())
    labs.append(y[:,0])


    

-----0/16----
-----1/16----
-----2/16----
-----3/16----
-----4/16----
-----5/16----
-----6/16----
-----7/16----
-----8/16----
-----9/16----
-----10/16----
-----11/16----
-----12/16----
-----13/16----
-----14/16----
-----15/16----


In [48]:
preds_all = np.concatenate(preds)
labs_all = np.concatenate(labs)

In [49]:
print(preds_all.shape)
print(labs_all.shape)

(2000, 1)
(2000,)


In [51]:
class_all = (preds_all[:,0] > 0.5)*1.0
print(np.mean(class_all == labs_all))

0.499


In [54]:
np.mean(labs_all)

0.501

In [ ]:
tf.saved_model.save(pretrained_model, mobilenet_save_path)


In [ ]:
distribution_results = est.evaluate(train_input_fn, steps=10)

In [ ]:
distribution_results

In [ ]:
train_pred = est.predict(train_input_fn)

In [1]:
train_pred

NameError: name 'train_pred' is not defined

In [9]:
counter = 0
for res in train_pred:
    if counter >10:
        break
    print(res)
    counter = counter + 1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/scratch/mmakar/dummy/model.ckpt-251
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'probabilities': array([0.], dtype=float32)}
{'classes': array([0.], dtype=float32), 'pr